## Ingesting PDF

In [1]:
%pip install --q unstructured langchain
%pip install --q "unstructured[all-docs]"

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.34.0 requires protobuf<5,>=3.20, but you have protobuf 5.28.2 which is incompatible.
tensorflow-intel 2.16.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.28.2 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader

In [3]:
local_path = "C:\\Users\\Administrator\\Downloads\\ollama_pdf_rag\\WEF_The_Global_Cooperation_Barometer_2024.pdf"
from langchain.document_loaders import PyPDFLoader

# Local PDF file uploads
if local_path:
  loader = PyPDFLoader(file_path=local_path)
  data = loader.load()
else:
  print("Upload a PDF file")

In [4]:
# Preview first page
data[0].page_content

'The Global Cooperation \nBarometer 2024\nINSIGHT REPORT\nJANUARY 2024In collaboration with \nMcKinsey & Company'

## Vector Embeddings

In [12]:
!python.exe -m pip install --upgrade pip

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
    --------------------------------------- 0.0/1.8 MB 220.2 kB/s eta 0:00:09
    --------------------------------------- 0.0/1.8 MB 220.2 kB/s eta 0:00:09
    --------------------------------------- 0.0/1.8 MB 179.6 kB/s eta 0:00:10
   - -------------------------------------- 0.1/1.8 MB 204.8 kB/s eta 0:00:09
   - -------------------------------------- 0.1/1.8 MB 204.8 kB/s eta 0:00:09
   - -------------------------------------- 0.1/1.8 MB 209.0 kB/s eta 0:00:09
   -- ------------------------------------- 0.1/1.8 MB 201.8 kB/s eta 0:00:09
   -- ------------------------------------- 0.1/1.8 MB 201.8 kB/s eta 0:00:09
   -- ------------------------------------- 0.1/1.8 MB 201.8 kB/s eta 0:00:09
   -- ----------------

In [13]:
!pip uninstall ollama

^C


In [14]:
!pip install ollama

In [17]:
!ollama --version

'ollama' is not recognized as an internal or external command,
operable program or batch file.


In [10]:
!pip install ollama


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
!ollama pull nomic-embed-text

'ollama' is not recognized as an internal or external command,
operable program or batch file.


In [9]:
!ollama list

'ollama' is not recognized as an internal or external command,
operable program or batch file.


In [18]:
%pip install --q chromadb
%pip install --q langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Building wheel for chroma-hnswlib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [5 lines of output]
      running bdist_wheel
      running build
      running build_ext
      building 'hnswlib' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for chroma-hnswlib
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (chroma-hnswlib)


Note: you may need to restart the kernel to use updated packages.


In [19]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [20]:
# Split and chunk 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

In [21]:
# Add to vector database
vector_db = Chroma.from_documents(
    documents=chunks, 
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    collection_name="local-rag"
)

ImportError: Could not import chromadb python package. Please install it with `pip install chromadb`.

## Retrieval

In [ ]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [ ]:
# LLM from Ollama
local_model = "mistral"
llm = ChatOllama(model=local_model)

In [ ]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [ ]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
chain.invoke(input(""))

 what is this about?


OllamaEmbeddings: 100%|███████████████████████████| 1/1 [00:00<00:00, 23.14it/s]


' This document is the Insight Report of The Global Cooperation Barometer 2024 by the World Economic Forum in collaboration with McKinsey & Company. It provides an analysis of the state of global cooperation across five pillars: trade and capital, innovation and technology, climate and natural capital, health and wellness, and peace and security. The report examines trends in cooperative actions and their outcomes to determine the overall level of global cooperation in each area. It also includes recommendations for leaders on how to reimagine global cooperation in a new era.'

In [ ]:
chain.invoke("What are the 5 pillars of global cooperation?")

OllamaEmbeddings: 100%|███████████████████████████| 1/1 [00:00<00:00, 51.85it/s]


' The 5 pillars of global cooperation are:\n\n1. Trade and capital\n2. Innovation and technology\n3. Climate and natural capital\n4. Health and wellness\n5. Peace and security.'

In [ ]:
# Delete all collections in the db
vector_db.delete_collection()